### Resampling landsat
Resample the landsat 30 * 30 m data to the NOAA LAI resolution (0.05 * 0.05). 
We use the reproject_rematch from rioxarray which it performs the reprojection and then take the area based average. 


In [1]:
# prepare the libraries and data paths
import xarray as xr
import numpy as np
import rioxarray
from rasterio.enums import Resampling
import glob
import pandas as pd
import time

dir = "/data/home/hamiddashti/hamid/nasa_above/greeness/"

in_dir = dir + ("data/processed_data/landsat/mosaic/")
out_dir = dir + "data/processed_data/landsat/resampled/"

target image is only a sample NOAA LAI

In [2]:
target_image = xr.open_dataset(
    dir + "data/processed_data/noaa_nc/lai_fapar/resampled/lai_monthly_resample_mean.nc"
)["LAI"].isel(time=5)
target_image = target_image.rio.write_crs("EPSG:4326").rename({"latitude":"y","longitude":"x"})

If out of memory use Dask

In [3]:
# Resampling TM5 data
for i in np.arange(1984, 2012):
    print("TM5 year:" + str(i))
    time.sleep(1)
    source_image = xr.open_dataarray(in_dir + "mosaic_NDVI_" + str(i) + "_TM5.tif",engine="rasterio")
    resampled = source_image.rio.reproject_match(
        target_image, resampling=Resampling.average
    )
    resampled.squeeze().to_netcdf(out_dir + "NDVI_resampled_" + str(i) + "_TM5.nc")
    del resampled

TM5 year:1984


In [ ]:
for i in np.arange(1999, 2014):
    print("ETM year:" + str(i))
    source_image = xr.open_dataarray(in_dir + "mosaic_NDVI_" + str(i) + "_ETM.tif",engine="rasterio")
    resampled = source_image.rio.reproject_match(
        target_image, resampling=Resampling.average
    )
    resampled.squeeze().to_netcdf(out_dir + "NDVI_resampled_" + str(i) + "_ETM.nc")
    del resampled

In [ ]:
# ds = (
#     xr.open_mfdataset(out_dir + "NDVI_*", concat_dim="time", combine="nested")
#     .drop("spatial_ref")
#     .to_array()
#     .squeeze()
#     .rename({"y": "latitude", "x": "longitude"})
# )
# times= pd.date_range("1984","2015",freq = "A")
# ds["time"] = times
# ds